In [3]:
import pandas as pd
import io
from tqdm import tqdm
from time import sleep
import numpy as np
import requests
import math
import json

In [4]:
filepath = '../data/NAD_r11.txt' # too large to push onto Git, but is part of the download from here: https://nationaladdressdata.s3.amazonaws.com/NAD_r12_TXT.zip

In [5]:
first_line = None

with open(filepath) as f:
    first_line = f.readline()

print(first_line)

OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,Bulk_Plus4,StN_PreMod,StN_PreDir,StN_PreTyp,StN_PreSep,StreetName,StN_PosTyp,StN_PosDir,StN_PosMod,AddNum_Pre,Add_Number,AddNum_Suf,LandmkPart,LandmkName,Building,Floor,Unit,Room,Addtl_Loc,Milepost,Longitude,Latitude,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired



In [6]:
total_lines = sum(1 for line in open(filepath))
print('Total number of lines: %s' % total_lines)

Total number of lines: 67357690


In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
def check_int(val):
    num = None
    try:
        num = str(int(float(val)))
    except Exception as e: print(e)
    return num

In [21]:
dfs = []

with tqdm(total=int(total_lines)) as pbar:
    with open(filepath) as f:
        first_line = f.readline() # skip first line
        i = 0
        j = 0
        lines = ''
        for line in f:
            pbar.update(1)
            if i% 10000 == 0:
                lines += line
                j+= 1
            if j >= 100:
                df = pd.read_csv(io.StringIO(first_line + lines), low_memory=False)
                dfs.append(df)
                j = 0
            i += 1
df = pd.concat(dfs)
df

100%|██████████████████████████████████████████████▉| 67357689/67357690 [00:36<00:00, 1847749.93it/s]


,OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,...,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired
0,-1,AZ,Coconino,SEDONA,NaN,NaN,SEDONA,86336.0,NaN,NaN,...,12SVD2942654247,{246F6382-AE25-4348-93AF-0ADCD0CCED00},Unknown,Unknown,State of Arizona,yavapai911.az.gov,NaN,4/7/2020 0:00:00,NaN,NaN
1,-1,AZ,Gila,PAYSON,NaN,NaN,PAYSON,85541.0,NaN,NaN,...,12SVC7141491028,{ABEB8A29-F297-47CF-B3F3-C341574FF716},Unknown,Unknown,State of Arizona,gila911.az.gov,NaN,8/4/2020 21:00:37,11/1/2016 16:01:30,NaN
2,-1,AZ,Gila,PAYSON,NaN,NaN,PAYSON,85541.0,NaN,NaN,...,12SVC7429791686,{C2539176-3BD8-4F22-B843-A9B0554FF774},Unknown,Unknown,State of Arizona,gila911.az.gov,NaN,8/4/2020 21:03:07,11/1/2016 16:02:02,NaN
3,-1,AZ,Gila,GLOBE,NaN,NaN,GLOBE,85501.0,NaN,NaN,...,12SWB1733197439,{30FFC6F5-37B2-415B-AF6E-F8E17F08E3B1},Unknown,Unknown,State of Arizona,gila911.az.gov,NaN,8/4/2020 21:05:31,11/1/2016 16:02:35,NaN
4,-1,AZ,Pinal,Unincorporated,SACATON,NaN,SACATON,85147.0,NaN,NaN,...,12SVB3061059408,{CDCA82B6-2BB7-4817-A850-CF53152DEE33},Unknown,Unknown,State of Arizona,gilariver911.az.gov,NaN,4/20/2022 0:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,-1,WY,Fremont,NaN,NaN,NaN,DUBOIS,82513.0,NaN,NaN,...,12TXP1074721887,{23E48061-C503-43B5-85B2-FA93FBAC0465},Unknown,Unknown,"Fremont County, Wyoming",DUBOIS,NaN,3/23/2017 0:00:00,NaN,NaN
6696,-1,WY,Campbell,NaN,NaN,NaN,Gillette,82718.0,NaN,NaN,...,13TDJ6569497543,{AF7B6882-0832-4BDF-A74E-7F9E7038E74F},Residential,Unknown,"Cambell County Public Works, Wyoming",CCGIS,NaN,10/5/2022 0:00:00,NaN,NaN
6697,-1,WY,Campbell,NaN,NaN,NaN,GILLETTE,82716.0,NaN,NaN,...,13TDK5818905709,{D1CBB9AC-8147-46CC-A546-1AB674A37055},Residential,Unknown,"Cambell County Public Works, Wyoming",COGGIS,NaN,10/5/2022 0:00:00,NaN,NaN
6698,-1,WY,Campbell,NaN,NaN,NaN,GILLETTE,82718.0,NaN,NaN,...,13TDK6572202244,{6F845F78-5364-44C5-B4A8-624EF178204E},Residential,Unknown,"Cambell County Public Works, Wyoming",COGGIS,NaN,10/5/2022 0:00:00,NaN,NaN


In [24]:
df[df.eq('DC').any(1)]

,OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,...,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired
5809,-1,DC,Washington,WASHINGTON,Glover Park,NaN,NaN,20007.0,NaN,NaN,...,18SUJ2004310413,{94620F3A-B812-446F-9874-4917D4BAAE04},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5810,-1,DC,Washington,WASHINGTON,Georgetown,025 E Georgetown,NaN,20007.0,NaN,NaN,...,18SUJ2104708392,{6FD02974-9DA7-4A82-AB30-E8E439867FC5},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5811,-1,DC,Washington,WASHINGTON,Eckington,019 A Eckington,NaN,20001.0,NaN,NaN,...,18SUJ2541409266,{26AD3CE8-F370-4409-B94A-521ECF49FA3B},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5812,-1,DC,Washington,WASHINGTON,Mt. Pleasant,036 A Mt. Pleasant,NaN,20010.0,NaN,NaN,...,18SUJ2342411696,{357248D4-E274-43CD-B966-0F21C4BB06DE},Multi,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5813,-1,DC,Washington,WASHINGTON,Southwest Waterfront,NaN,NaN,20024.0,NaN,NaN,...,18SUJ2468205269,{5930AF6D-2D64-4D2F-8B1A-660FB1435545},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,-1,DC,Washington,WASHINGTON,Burleith,008 B Burleith,NaN,20007.0,NaN,NaN,...,18SUJ1992009174,{4A57E0A8-F1F1-41B9-9603-5F4D8C8A7EC4},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5847,-1,DC,Washington,WASHINGTON,American University,001 C American University,NaN,20016.0,NaN,NaN,...,18SUJ1982712681,{0361DA81-9D6C-43C0-AEB8-269B5CA8751C},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5848,-1,DC,Washington,WASHINGTON,Mt. Pleasant,036 A Mt. Pleasant,NaN,20010.0,NaN,NaN,...,18SUJ2321710930,{45E6382A-8740-4FF2-885A-5C341B6CD657},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5849,-1,DC,Washington,WASHINGTON,Georgetown,025 I Georgetown,NaN,20007.0,NaN,NaN,...,18SUJ2115008159,{28B9104A-7A66-4125-90FA-FD1AE62F3BDF},Unknown,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN


In [16]:
pdf = df.sample(6000)

In [27]:
'DC' in df['State'].unique()

True

In [38]:
dc = df[df['State'] == 'DC']
dc

,OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,...,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired
5809,-1,DC,Washington,WASHINGTON,Glover Park,NaN,NaN,20007.0,NaN,NaN,...,18SUJ2004310413,{94620F3A-B812-446F-9874-4917D4BAAE04},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5810,-1,DC,Washington,WASHINGTON,Georgetown,025 E Georgetown,NaN,20007.0,NaN,NaN,...,18SUJ2104708392,{6FD02974-9DA7-4A82-AB30-E8E439867FC5},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5811,-1,DC,Washington,WASHINGTON,Eckington,019 A Eckington,NaN,20001.0,NaN,NaN,...,18SUJ2541409266,{26AD3CE8-F370-4409-B94A-521ECF49FA3B},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5812,-1,DC,Washington,WASHINGTON,Mt. Pleasant,036 A Mt. Pleasant,NaN,20010.0,NaN,NaN,...,18SUJ2342411696,{357248D4-E274-43CD-B966-0F21C4BB06DE},Multi,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5813,-1,DC,Washington,WASHINGTON,Southwest Waterfront,NaN,NaN,20024.0,NaN,NaN,...,18SUJ2468205269,{5930AF6D-2D64-4D2F-8B1A-660FB1435545},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,-1,DC,Washington,WASHINGTON,Burleith,008 B Burleith,NaN,20007.0,NaN,NaN,...,18SUJ1992009174,{4A57E0A8-F1F1-41B9-9603-5F4D8C8A7EC4},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5847,-1,DC,Washington,WASHINGTON,American University,001 C American University,NaN,20016.0,NaN,NaN,...,18SUJ1982712681,{0361DA81-9D6C-43C0-AEB8-269B5CA8751C},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5848,-1,DC,Washington,WASHINGTON,Mt. Pleasant,036 A Mt. Pleasant,NaN,20010.0,NaN,NaN,...,18SUJ2321710930,{45E6382A-8740-4FF2-885A-5C341B6CD657},Residential,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN
5849,-1,DC,Washington,WASHINGTON,Georgetown,025 I Georgetown,NaN,20007.0,NaN,NaN,...,18SUJ2115008159,{28B9104A-7A66-4125-90FA-FD1AE62F3BDF},Unknown,Unknown,OCTO DATA TEAM District of Columbia,DCRA,NaN,10/31/2022 0:00:00,NaN,NaN


In [30]:
dc['Zip_Code'].unique()

array([20007., 20001., 20010., 20024., 20008., 20037., 20005., 20019.,
       20009., 20032., 20016., 20052., 20020., 20003., 20015., 20018.])

In [37]:
def clean_df(dc):
    def check_int(val):
        num = None
        try:
            num = str(int(float(val)))
        except Exception as e: print(e)
        return num

    dc = dc.replace('nan', '')
    dc = dc.fillna('')   
    dc = dc[dc['Zip_Code'] != 'nan']
    dc = dc[dc['Zip_Code'] != '']
    dc = dc.apply(lambda x: x.astype(str).str.lower()) # convert everything lower case

    # cleaning zip code
    dc['Zip_Code'] = dc['Zip_Code'].apply(lambda x: check_int(x))

    # removing invalid zip rows
    dc = dc[dc['Zip_Code'].notnull()]
    return dc

In [39]:
dc = clean_df(dc)
dc

,OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,...,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired
5809,-1,dc,washington,washington,glover park,,,20007,,,...,18suj2004310413,{94620f3a-b812-446f-9874-4917d4baae04},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5810,-1,dc,washington,washington,georgetown,025 e georgetown,,20007,,,...,18suj2104708392,{6fd02974-9da7-4a82-ab30-e8e439867fc5},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5811,-1,dc,washington,washington,eckington,019 a eckington,,20001,,,...,18suj2541409266,{26ad3ce8-f370-4409-b94a-521ecf49fa3b},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5812,-1,dc,washington,washington,mt. pleasant,036 a mt. pleasant,,20010,,,...,18suj2342411696,{357248d4-e274-43cd-b966-0f21c4bb06de},multi,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5813,-1,dc,washington,washington,southwest waterfront,,,20024,,,...,18suj2468205269,{5930af6d-2d64-4d2f-8b1a-660fb1435545},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,-1,dc,washington,washington,burleith,008 b burleith,,20007,,,...,18suj1992009174,{4a57e0a8-f1f1-41b9-9603-5f4d8c8a7ec4},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5847,-1,dc,washington,washington,american university,001 c american university,,20016,,,...,18suj1982712681,{0361da81-9d6c-43c0-aeb8-269b5ca8751c},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5848,-1,dc,washington,washington,mt. pleasant,036 a mt. pleasant,,20010,,,...,18suj2321710930,{45e6382a-8740-4ff2-885a-5c341b6cd657},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
5849,-1,dc,washington,washington,georgetown,025 i georgetown,,20007,,,...,18suj2115008159,{28b9104a-7a66-4125-90fa-fd1ae62f3bdf},unknown,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,


In [49]:
sample = dc.sample(1)

sample.describe()

,OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,...,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired
count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,-1,dc,washington,washington,cleveland park,,,20016,,,...,18suj2008711926,{2102e993-f7e5-413c-966c-1f28e52f2f55},residential,unknown,octo data team district of columbia,dcra,,10/31/2022 0:00:00,,
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
